In [11]:
import numpy as np
from read_write_mrc import read_mrc,write_mrc 
from utils import *
import os
import gc

In [ ]:
img_directory = '../NewCellsUnsegmentedCells/Unsegmented_5minNewCells(Plus 25mMGlu )/MrcFormat_tomograms'
mask_directory = '../NewCellsUnsegmentedCells/5min_CellMaskTogether/MrcFormat_membraneMasks/'

Outimg_path = 'slices/New5minCellsProcessedMrc/'
Outmask_path = 'slices/New5minCellsProcessedMask/'

os.makedirs(Outimg_path,exist_ok=True)
os.makedirs(Outmask_path,exist_ok=True)

img_name_list = os.listdir(img_directory)
mask_name_list =  os.listdir(mask_directory)

img_name_list.sort()
mask_name_list.sort()

target_size = (704,704,704)
count=0
skipped = 0
for img_name,mask_name in zip(img_name_list, mask_name_list):
    
    # print(img_name[0:-4])
    # print(mask_name[0:-11])
    
    # assert str(img_name[0:-4]+str('_labels.mrc')) == str(mask_name), "Mask and image don't match."
    
    img_path = os.path.join(img_directory, img_name)
    mask_path = os.path.join(mask_directory, mask_name)
    
    img = read_mrc(img_path).get()
    mask = read_mrc(mask_path).get()
    assert img.shape == mask.shape, "Mask and image don't match."
    
    if any(dim > 704 for dim in img.shape):
        print(f"Skipping {img_name} (shape: {img.shape}) — exceeds 704 in one or more dimensions.")
        skipped += 1
        continue
    
    imgpp2 = ves_mito_separator(img)
    imgpp2 = mask_crop(imgpp2,mask)
    padded_imgpp2 = center_padding(imgpp2,target_size)
    padded_mask = center_padding(mask,target_size)
    
    assert padded_imgpp2.shape == (704,704,704), f"Something error after padding in padded_imgpp2 {padded_imgpp2.shape}"
    assert padded_mask.shape == (704,704,704), f"Something error after padding in padded_mask"
    gc.collect()
    write_mrc(Outimg_path+img_name[0:-4]+'.mrc', padded_imgpp2.astype('float32'))
    write_mrc(Outmask_path+img_name[0:-4]+'_labels.mrc', padded_mask.astype('int8'))
    count+=1
    print(count ,end='\r')
print(f'total skipped images {skipped}')